In [1]:
from utils.spark_utils import get_spatial_spark_session

In [5]:
# spark = get_spatial_spark_session()
CENS_STAGING_PATH = "/mnt/c/wsl_transfers/cens_staging"

In [10]:
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
!jupyter nbextension enable hinterland/hinterland

Enabling notebook extension hinterland/hinterland...
      - Validating: OK


## Creating ETL for postcode poly, to write into new lad partitions; 

In [44]:
import duckdb
from utils.dbutils import sdf_df_from_query

In [59]:
con = duckdb.connect("/home/basal/live_datasets/processed/cens.db", read_only = True)
pc_mapping_pldf = con.execute("select pcd, ltla22cd as lad_cd, oa21cd as oa_cd from pc_to_oa_mapping").pl()
con.close()

In [82]:
pc_mapping_pldf_pdf = pc_mapping_pldf.select([
    pl.col('pcd').str.to_lowercase()\
        .str.replace_all(r"[&,-,(,),:,']", '')\
        .str.replace_all(r'\s', '').alias("area_cd"),
    pl.col('lad_cd'),
    pl.col('oa_cd')
]).to_pandas()

In [90]:
pc_mapping_pldf_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352552 entries, 0 to 2352551
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   area_cd  object
 1   lad_cd   object
 2   oa_cd    object
dtypes: object(3)
memory usage: 53.8+ MB


In [83]:
pc_mapping_sdf = spark.createDataFrame(pc_mapping_pldf_pdf)

/home/basal/envs/dbc1/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [85]:
pc_poly = spark.read.parquet(f"/mnt/c/Users/lohith/Downloads/OS_data_sets/processed/postcode_polygons").selectExpr(
        "lower(postcode) as area_cd",
        "ST_FlipCoordinates(postcode_poly_4326) as poly_4326_fc"
    )

In [86]:
pc_mapping_sdf, pc_poly

(DataFrame[area_cd: string, lad_cd: string, oa_cd: string],
 DataFrame[area_cd: string, poly_4326_fc: udt])

## Create GPD

In [2]:
import glob
import os 
import geopandas as gpd
CENS_STAGING_PATH = "/mnt/c/wsl_transfers/cens_staging"

file_list = glob.glob(f"{CENS_STAGING_PATH}/geo_parquets/pc/*")


In [7]:
pc_gpd = gpd.GeoDataFrame(pd.concat(pc_gpd_all, ignore_index=False) )

In [8]:
pc_gpd

,poly_4326_fc
area_cd,
ts225fz,"POLYGON ((-1.33952 54.64778, -1.33950 54.64778..."
ts225gh,"POLYGON ((-1.35373 54.63985, -1.35406 54.63985..."
ts225gt,"POLYGON ((-1.34697 54.64573, -1.34677 54.64579..."
ts225gt,"POLYGON ((-1.34218 54.64627, -1.34218 54.64627..."
ts225gu,"POLYGON ((-1.34317 54.64570, -1.34319 54.64570..."
...,...
cf484td,"POLYGON ((-3.33459 51.67914, -3.33444 51.67917..."
cf484tf,"POLYGON ((-3.34781 51.69711, -3.34784 51.69711..."
cf484tj,"POLYGON ((-3.34731 51.70116, -3.34726 51.70146..."


In [9]:
# %%timeit
points_series = gpd.read_parquet('/mnt/c/wsl_transfers/cens_staging/gpd_parquets/pc/E06000002.parquet')[:400]['poly_4326_fc'].centroid
points_df = gpd.GeoDataFrame(geometry=gpd.GeoSeries(points_series))

In [10]:
# %%timeit
pc_gpd.sjoin(
    points_df,
    how="right",
    predicate="contains"
)

,index_left,geometry
area_cd,,
ts11aa,ts11aa,POINT (-1.24487 54.57578)
ts11af,ts11su,POINT (-1.24018 54.57745)
ts11jh,ts11jh,POINT (-1.23192 54.57655)
ts11jt,ts11jt,POINT (-1.23355 54.57688)
ts11le,ts11le,POINT (-1.23814 54.57665)
...,...,...
ts39es,ts39es,POINT (-1.18295 54.56740)
ts39ex,ts39ex,POINT (-1.18965 54.56442)
ts39ex,ts39ex,POINT (-1.18832 54.56455)


In [11]:
lad = gpd.read_parquet('/mnt/c/wsl_transfers/cens_staging/gpd_parquets/lad_gpd.parquet')

None


In [25]:
points_series.within(lad['poly_4326_fc'])

/home/basal/envs/dbc1/lib/python3.8/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


area_cd
E06000001    False
E06000002    False
E06000003    False
E06000004    False
E06000005    False
             ...  
ts39es       False
ts39ex       False
ts39ex       False
ts39he       False
ts39hn       False
Length: 774, dtype: bool

## Find postcode for a point

In [76]:
lad_gpd = gpd.read_parquet('/mnt/c/wsl_transfers/cens_staging/gpd_parquets/lad_gpd.parquet')

In [73]:
lat, lon = 51.573404, 0.083952
df = pd.DataFrame([{'lon':lon, 'lat':lat}])
df['coords'] = list(zip(df['lon'],df['lat']))
df['coords'] = df['coords'].apply(Point)
point_gdf = gpd.GeoDataFrame(df, geometry='coords')
point_gdf



,lon,lat,coords
0,0.083952,51.573404,POINT (0.08395 51.57340)


In [109]:
def pcd_from_point(lat, lon):
    # Create point geom df
    df = pd.DataFrame([{'lon':lon, 'lat':lat}])
    df['coords'] = list(zip(df['lon'],df['lat']))
    df['coords'] = df['coords'].apply(Point)
    point_gdf: gpd.GeoDataFrame = gpd.GeoDataFrame(df, geometry='coords')
    # find the lad_cd required for pc lookup
    _lad_gpd = lad_gpd.sjoin(
        point_gdf,
        how="inner",
        predicate="contains"
    )
    lad_cd = _lad_gpd.index[0]
    # find postcode using lad_cd as lookup
    pc_gpd = gpd.read_parquet(f'/mnt/c/wsl_transfers/cens_staging/gpd_parquets/pc/{lad_cd}.parquet').sjoin(
        point_gdf,
        how="inner",
        predicate="contains"
    )
    return pc_gpd.index[0]

In [115]:
lat, lon = 53.38542828811447, -2.592789058525886
pcd_from_point(lat, lon)

'wa11hn'

In [ ]:
crime_point_index_gpd_w_pc.index.name = 'pcd'

In [ ]:
crime_point_index

,oa21cd,lad22cd,lat,lon,lsoa11cd,geometry
0,E00062396,E06000006,53.371119,-2.719657,E01012370,POINT (-2.71966 53.37112)
1,E00062396,E06000006,53.372592,-2.719833,E01012368,POINT (-2.71983 53.37259)
2,E00062396,E06000006,53.368875,-2.719139,E01012370,POINT (-2.71914 53.36888)
3,E00062396,E06000006,53.371074,-2.719597,E01012370,POINT (-2.71960 53.37107)
4,E00062396,E06000006,53.370604,-2.721513,E01012370,POINT (-2.72151 53.37060)
...,...,...,...,...,...,...
1283046,W00009924,W06000018,51.651698,-3.136300,W01001327,POINT (-3.13630 51.65170)
1283047,W00009924,W06000018,51.654022,-3.146925,W01001327,POINT (-3.14692 51.65402)
1283048,W00009924,W06000018,51.654017,-3.146549,W01001327,POINT (-3.14655 51.65402)
1283049,W00009924,W06000018,51.656464,-3.137128,W01001327,POINT (-3.13713 51.65646)


In [63]:
crime_point_index_gpd_w_pc.reset_index()[['pcd', 'oa21cd', 'lad22cd',]]

,pcd,oa21cd,lad22cd,lat,lon,lsoa11cd
0,wa44bt,E00062486,E06000006,53.330608,-2.647170,E01012385
1,wa44fx,E00062486,E06000006,53.338810,-2.634619,E01012385
2,wa44ge,E00062486,E06000006,53.331937,-2.642461,E01012385
3,wa46nq,E00062481,E06000006,53.347795,-2.647160,E01012386
4,wa46nq,E00062481,E06000006,53.348094,-2.648262,E01012386
...,...,...,...,...,...,...
95,wa89la,E00062524,E06000006,53.381986,-2.728575,E01012394
96,wa89la,E00062524,E06000006,53.382518,-2.729757,E01012394
97,wa89lj,E00062524,E06000006,53.381586,-2.730763,E01012394
98,wa89lj,E00062524,E06000006,53.381640,-2.730779,E01012394
